<a href="https://colab.research.google.com/github/gurucl/apache_beam_tutorial/blob/main/Apache_Beam_Workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello world!')

In [ ]:
! pip3 install apache_beam

In [ ]:
! pwd
! ls sample_data

In [ ]:
import apache_beam as beam
p1 = beam.Pipeline()
output = ( p1 
          | beam.io.ReadFromText("/content/sample_data/tiny_movies.csv", skip_header_lines=1)
          | beam.Map(lambda x: x.split(","))
       #   | beam.Sample.FixedSizeGlobally(5)
         | beam.Filter(lambda x: float(x[3].strip()) > 8.8)
        # | beam.io.WriteToText("sample_data/output", "result", num_shards=3, header = "Movie ID, Movie_Name, Year, Rating", compression_type = beam.io.filesystem.CompressionTypes.GZIP )
        # | beam.Map(print)
          )
p1.run()

In [ ]:
import apache_beam as beam
p1 = beam.Pipeline()

schema1 = """ {
   "type" : "record",
   "namespace" : "Tutorialspoint",
   "name" : "Employee",
   "fields" : [
      { "name" : "Movie_id" , "type" : "string" },
      { "name" : "Name" , "type" : "string" },
      { "name" : "Year" , "type" : "string" },
      { "name" : "Rating" , "type" : "string" }
   ]
}"""
output = ( p1 
          | beam.io.ReadFromText("/content/sample_data/tiny_movies.csv", skip_header_lines=1)
          | beam.Map(lambda x: x.split(","))
       #   | beam.Sample.FixedSizeGlobally(5)
         | beam.Filter(lambda x: float(x[3].strip()) > 8.8)
        # | beam.io.WriteToText("sample_data/output", "result", shard_name_template='-SS-NN', num_shards=3, header = "Movie ID, Movie_Name, Year, Rating", )
          | beam.io.WriteToAvro("out", schema = schema1)
        # | beam.Map(print)
          )
p1.run()

In [ ]:
! cat sample_data/tiny_movies.csv

In [ ]:
import apache_beam as beam

p1 = beam.Pipeline()

result = (p1
        | beam.Create([1,2,3,4,5,6,7,8,9,10])
      #   | beam.Filter(lambda x: x%2==0)
       | beam.Map(print)
          )
p1.run()



In [52]:
import apache_beam as beam
import re

p1 = beam.Pipeline()

list = ["K", "G", "F", "Kolar", "and", "to"]

def get_details(str):
  return True if str in list else False
  # if str in list:
  #   return True

  
  

result = (
    p1
    | beam.io.ReadFromText("/content/sample_data/kgf_2.txt")
    | beam.FlatMap(lambda x: re.findall('[a-zA-Z]+', x))
    | beam.Filter(get_details)
    | beam.Map(lambda x: (x,1))
    | beam.CombinePerKey(sum)
    # | beam.Map(print)
    | beam.io.WriteToText("/content/sample_data/words_output")
    
)
p1.run()



In [63]:
import apache_beam as beam

p1 = beam.Pipeline()

movies_data = [
               (1,"Avenger"),
               (2,"Infinity War"),
               (3,"Winter Soldier"),
               (4,"Thor - dark World")
]
ratings_data = [
                (1, 8.5),
                (2, 9.1),
                (3, 7.5),
                (1, 8.9),
                (2, 9.2) 
]

movies = p1 |'reading Movies data' >> beam.Create(movies_data)
ratings = p1 |'reading Ratings data' >> beam.Create(ratings_data)

# joined_data = ( {'movie_name':movies, 'movie_ratings':ratings} 
#                |  beam.CoGroupByKey())  | beam.Map(print)
#              #  | beam.Map(lambda x: (x[0], x[1][1][0], x[1][0]))
              
joined_data = ( {movies, ratings} 
               |  beam.CoGroupByKey()) | beam.Map(lambda x: (x[0], x[1][1][0], x[1][0], sum(x[1][0]), sum(x[1][0])/len(x[1][0]) if len(x[1][0])!=0 else 0 ))| beam.Map(print)

p1.run()

(1, 'Avenger', [8.5, 8.9], 17.4, 8.7)
(2, 'Infinity War', [9.1, 9.2], 18.299999999999997, 9.149999999999999)
(3, 'Winter Soldier', [7.5], 7.5, 7.5)
(4, 'Thor - dark World', [], 0, 0)


In [70]:
import apache_beam as beam


p1 = beam.Pipeline()

def my_partition_fn(x, num_partitions):
  return 0 if x%2==0 else 1

result = (p1
          | beam.Create(range(1,11))
          | beam.Partition(my_partition_fn, 2)
        )
result[1] | beam.Map(print)

p1.run()

1
3
5
7
9


In [77]:
import apache_beam as beam

p1 = beam.Pipeline()

odd = [1,3,5,7,9]
even = [2,4,6,8]
name = ['Ram', 'Rahim', 'Robert' ]

odd_p1 = (p1| 'creating Odd PCollection' >> beam.Create(odd))
even_p1 = (p1| 'creating Even PCollection' >> beam.Create(even))
name_p1 = (p1| 'creating Name PCollection' >> beam.Create(name))

( {odd_p1, even_p1, name_p1} | beam.Flatten()) | beam.Map(print)

p1.run()




1
3
5
7
9
2
4
6
8
Ram
Rahim
Robert
